In [1]:
from pathlib import Path 
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import date


sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm0104",
    security_token="EwVPSL02qXM8XQ2pkUdA13aW",
)

In [2]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
    ]
]
Empresas_SF.shape

(7947, 7)

In [3]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(7951, 5)

In [4]:
#Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A"
    ]
]

Programas_empresas.shape

(7955, 9)

fin empresas y comienzo de rescate de pacientes


In [5]:
#CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        'Phone'
        
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape

(1561250, 9)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.

In [6]:
#buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
    columns={
        "Id": "Id Pacientes_Programas__c",
        "Paciente__c": "Id Account",
        "Programas_por_Empresas__c": "Programas_por_Empresas__c",
        "padre__c": "Nombre Campaña"        
    }
)
Paciente_Programas_SF.shape

(1276080, 4)

In [7]:
#Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_empresas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c", 
)
empresas_unidas.shape

(1276080, 12)

Carga de Archivo carga

In [20]:
nuevos = ("F:/Empresas SF/reportes/carga.xlsx")
campaña = 'Pesco'
Pais = 'Chile'




Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].str.replace("-", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head()                                       

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_2880\4244714134.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")


,rut1,Estado,Empleado - Nombre Completo,apellidos,nombres,Empresa - Nombre Empresa,Trabajo - Cargo,Trabajo - Nombre Área,Trabajo - Nombre Sub-área Asignada(o),Empleado - Estado,Trabajo - Fecha Ingreso Compañía,Trabajo - Centro de Costo,Trabajo - Nombre Supervisor,Empleado - Fecha de Nacimiento,sexo,Empleado - Nacionalidad,Empleado - Teléfono Particular,email,rut
0,12857704-1,12857704-1,Narvaez Molina Rodrigo,Narvaez Molina,Rodrigo,Peña Spoerer y Cía SA,JEFE DE SUCURSAL,09 Sucursal Antofagasta,Servicios Post Venta,Activo,2020-03-02,428,Jaime Eduardo Andrews Soto,1975-04-17,M,Chilena,9.991902e+08,narvaezrodrigo@hotmailcom,128577041
1,17253870-3,17253870-3,Steck Silva Stephanie Francesca,Steck Silva,Stephanie Francesca,Peña Spoerer y Cía SA,EJECUTIVO COMERCIAL,04 COMERCIAL,EQ ESPECIALES - MANEJO RESIDUOS,Activo,2021-05-03,40411,Antonio Del Rio Hoffmann,1989-10-15,F,Chilena,5.699477e+10,stephaniesteck@hotmailcom,172538703
2,14124888-k,14124888-k,Zuñiga Quiroz Gloria Macarena,Zuñiga Quiroz,Gloria Macarena,Peña Spoerer y Cía SA,SUPERVISOR_,02 ADMINISTRACION Y FINANZAS,CONTABILIDAD,Activo,2021-05-17,203,Mario Antonio Contreras Fuentes,1981-01-22,F,Chilena,5.699837e+10,GLORIA_MZQ@HOTMAILCOM,14124888K
3,18508342-k,18508342-k,Espejo Castillo Jose Armando,Espejo Castillo,Jose Armando,Peña Spoerer y Cía SA,TECNICO DE SERVICIO,09 Sucursal Antofagasta,Taller Servicios,Activo,2021-01-11,301,Rodrigo Narvaez Molina,1993-12-22,M,Chilena,9.663543e+08,joseespejo221293@gmailcom,18508342K
4,16574011-4,16574011-4,Vidal Osorio Leonora Paz,Vidal Osorio,Leonora Paz,Peña Spoerer y Cía SA,ANALISTA CONTABLE SENIOR,02 ADMINISTRACION Y FINANZAS,CONTABILIDAD,Activo,2021-06-14,203,Gloria Macarena Zuñiga Quiroz,1987-02-17,F,Chilena,5.694451e+10,vidalleonora@gmailcom,165740114


In [21]:
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
#Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace(
    "-", ""
)


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(
    Pacientes_SF["Rut"])

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_2880\4159893363.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_2880\4159893363.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [22]:
#aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
Pacientes_con_id.shape

(259, 29)

In [23]:
Pacientes_con_id.head()

,rut1,Estado,Empleado - Nombre Completo,apellidos,nombres,Empresa - Nombre Empresa,Trabajo - Cargo,Trabajo - Nombre Área,Trabajo - Nombre Sub-área Asignada(o),Empleado - Estado,...,RutEstaoNo,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono
0,12857704-1,12857704-1,Narvaez Molina Rodrigo,Narvaez Molina,Rodrigo,Peña Spoerer y Cía SA,JEFE DE SUCURSAL,09 Sucursal Antofagasta,Servicios Post Venta,Activo,...,True,0018c00002egsNOAAY,128577041,Rodrigo Narvaez Molina,RUT,None,None,None,Hombre,None
1,17253870-3,17253870-3,Steck Silva Stephanie Francesca,Steck Silva,Stephanie Francesca,Peña Spoerer y Cía SA,EJECUTIVO COMERCIAL,04 COMERCIAL,EQ ESPECIALES - MANEJO RESIDUOS,Activo,...,True,0018c00002egsNQAAY,172538703,Stephanie Francesca Steck Silva,RUT,None,None,None,Mujer,None
2,14124888-k,14124888-k,Zuñiga Quiroz Gloria Macarena,Zuñiga Quiroz,Gloria Macarena,Peña Spoerer y Cía SA,SUPERVISOR_,02 ADMINISTRACION Y FINANZAS,CONTABILIDAD,Activo,...,True,0018c00002amLUtAAM,14124888K,Gloria Macarena Zuñiga Quiroz,RUT,None,None,None,Mujer,998369653
3,18508342-k,18508342-k,Espejo Castillo Jose Armando,Espejo Castillo,Jose Armando,Peña Spoerer y Cía SA,TECNICO DE SERVICIO,09 Sucursal Antofagasta,Taller Servicios,Activo,...,True,0018c00002egsMmAAI,18508342K,Jose Armando Espejo Castillo,RUT,None,None,None,Otros,None
4,16574011-4,16574011-4,Vidal Osorio Leonora Paz,Vidal Osorio,Leonora Paz,Peña Spoerer y Cía SA,ANALISTA CONTABLE SENIOR,02 ADMINISTRACION Y FINANZAS,CONTABILIDAD,Activo,...,True,0018c00002amT5jAAE,165740114,Leonora Vidal Osorio,RUT,None,None,None,Mujer,None


In [24]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx",index=False)

In [25]:
# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = Pacientes_con_id["rut"]
if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = (
        Pacientes_con_id["email"].astype(str).str.lower().replace("nan", "")
    )
if ("nacimiento" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"]
if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "M": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "O",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "F ": "Mujer",
        
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = Pacientes_con_id["celular"]

Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"
Subir_pacientes["BillingCountry"] = "Chile"
Subir_pacientes["Identification_Type__c"] = "RUT"


if Pais == 'Chile':    
    Subir_pacientes['Pa_s_del_Paciente__c'] = 'a1U8c000005dAtnEAE'
else:
    Subir_pacientes['Pa_s_del_Paciente__c'] = 'a1U8c000004cvS7EAI'


Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]


# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account.xlsx", index=False)
Subir_pacientes.head()

,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonEmail,HealthCloudGA__Gender__pc,RecordTypeId,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c
229,False,Jose Gerardo,Colmenares Gomez,267101418,jgcg55@gmailcom,Hombre,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
231,False,Maria Cristina,Ponce Morales,12721798K,mcponcemo@gmailcom,Mujer,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
232,False,Maryorie Carolina,Olguin Molina,150995736,maryorie_olguin@yahoocom,Mujer,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
243,False,Brandon Bruce,Gertner Huenupi,201038642,brandongertner@gmailcom,Hombre,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
245,False,Wilson Federico,Dunarri Torres,193043399,dunarriwilson7@gmailcom,Hombre,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE


In [26]:
    
clinica_virtual = pd.DataFrame()    
clinica_virtual['rut'] = Pacientes_con_id["rut1"].str.upper()
clinica_virtual['firstName'] = Pacientes_con_id['nombres']
clinica_virtual['lastName'] = Pacientes_con_id['apellidos']
clinica_virtual['accountId'] = campaña



Codigos_Clinica_Virtual = {
'BCI':'1',
'Farmacias Ahumada':'2',
'MetLife':'3',
'Vida Security':'4',
'Willis':'5',
'Entel':'6',
'Testing':'7',
'3SP':'8',
'HomeCare':'9',
'Pedro de Valdivia':'10',
'Colaboradores':'11',
'MineraAbra':'12',
'CMPC':'13',
'Lipigas':'14',
'Asismed':'15',
'Siempre Perú':'16',
'Farmacias Cruz Verde':'17',
'WFS Food Services':'18',
'VidaCel':'19',
'Dermik':'20',
'EY':'21',
'IND':'22',
'Sindicato Walmart':'23',
'Instruvalve':'24',
'VidaCel Colaboradores':'25',
'Socofar':'26',
'Milan Fajanovic':'27',
'THBseguros':'28',
'Pesco':'29',
'Burgos - Consorcio':'30',
'COFAMA-SILENTIUM':'31',
'KAUFMANN':'32',
'QuePlan':'33',
'Zurich':'34',
'Colaboradores Siempre Chile':'35',
'TALO':'36',
'TRANSVIP':'37',
'Vida Security Colaboradores':'38',
'FORUS':'39',
'SURA':'40'
}

clinica_virtual.replace({"accountId": Codigos_Clinica_Virtual}, inplace=True)
clinica_virtual.to_excel("F:/Empresas SF/reportes/Clinica_Virtual.xlsx", index=False)

In [27]:
campaña_seleccionada = empresas_unidas[empresas_unidas["Campaña"] == campaña]
campaña_seleccionada.head()


,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Codigo EMP,Codigo A
215053,a1Y8c00000C6VeNEAV,0018c00002amRRLAA2,a1W8c0000083iWjEAI,Pesco,0018c00002cXJTtAAO,Pesco Holding Pesco Peña Spoerer Y Cia S.A.,Holding Pesco,Pesco,None,None,EMP7399,A-0723
215054,a1Y8c00000C6VeOEAV,0018c00002amRzAAAU,a1W8c0000083iWjEAI,Pesco,0018c00002cXJTtAAO,Pesco Holding Pesco Peña Spoerer Y Cia S.A.,Holding Pesco,Pesco,None,None,EMP7399,A-0723
215055,a1Y8c00000C6VePEAV,0018c00002amLUtAAM,a1W8c0000083iWjEAI,Pesco,0018c00002cXJTtAAO,Pesco Holding Pesco Peña Spoerer Y Cia S.A.,Holding Pesco,Pesco,None,None,EMP7399,A-0723
215056,a1Y8c00000C6VeQEAV,0018c00002amSFBAA2,a1W8c0000083iWjEAI,Pesco,0018c00002cXJTtAAO,Pesco Holding Pesco Peña Spoerer Y Cia S.A.,Holding Pesco,Pesco,None,None,EMP7399,A-0723
215057,a1Y8c00000C6VeREAV,0018c00002amSFFAA2,a1W8c0000083iWjEAI,Pesco,0018c00002cXJTtAAO,Pesco Holding Pesco Peña Spoerer Y Cia S.A.,Holding Pesco,Pesco,None,None,EMP7399,A-0723


In [28]:
campaña_seleccionada.shape

(165, 12)

In [29]:

if campaña != 'MetLife':
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente"
    )
    pacientes_en_campaña['Paciente__c'] = 'Id Pacientes'
    pacientes_en_campaña['Programas_por_Empresas__c'] = 'Id Programas Por empresa'
    pacientes_en_campaña.to_excel("F:/Empresas SF/reportes/campaña_empresa.xlsx",index=False)

In [30]:
campaña_en_pacientes = pd.merge(
    left=Pacientes_con_id,
    right=campaña_seleccionada,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account"
)
campaña_en_pacientes.to_excel("F:/Empresas SF/reportes/archivo_campaña.xlsx",index=False)